In [3]:
!pip install ultralytics --upgrade --quiet

In [2]:
from ultralytics import YOLO

In [3]:
import os
import cv2
import numpy as np
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm

In [4]:
Image_dir = '/home/temgar.s/pneumonia_dataset/Pneumonia/Training/Images'
Mask_dir = '/home/temgar.s/pneumonia_dataset/Pneumonia/Training/Masks'

In [5]:
output_img_dir = 'Pneumonia-YOLO/images/train'
output_label_dir = 'Pneumonia-YOLO/labels/train'

In [6]:
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

In [7]:
def get_yolo_bbox_from_mask(mask):
    mask_np = np.array(mask)
    if mask_np.max() == 0:
        return None
    coords = cv2.findNonZero(mask_np)
    x, y, w, h = cv2.boundingRect(coords)
    H, W = mask_np.shape
    # Normalize bbox
    x_center = (x + w/2) / W
    y_center = (y + h/2) / H
    w_norm = w / W
    h_norm = h / H
    return (0, x_center, y_center, w_norm, h_norm)  # class 0

for file in tqdm(os.listdir(Image_dir)):
    if not file.endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(Image_dir, file)
    mask_path = os.path.join(Mask_dir, file)

    # Save image
    img_out_path = os.path.join(output_img_dir, file)
    Image.open(img_path).convert("RGB").save(img_out_path)

    # Process mask
    mask = Image.open(mask_path).convert('L').resize((256, 256))
    bbox = get_yolo_bbox_from_mask(mask)

    label_file = os.path.join(output_label_dir, file.rsplit('.', 1)[0] + '.txt')
    with open(label_file, 'w') as f:
        if bbox:
            f.write(' '.join(map(str, bbox)) + '\n')


100%|██████████| 26684/26684 [2:40:40<00:00,  2.77it/s]  


In [6]:
%%writefile yolo_detection.py

import os
import cv2
import numpy as np
from PIL import Image, ImageFile, ImageDraw
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

Image_dir = '/home/temgar.s/pneumonia_dataset/Pneumonia/Training/Images'
Mask_dir = '/home/temgar.s/pneumonia_dataset/Pneumonia/Training/Masks'
output_img_dir = 'Pneumonia-YOLO/images/train'
output_label_dir = 'Pneumonia-YOLO/labels/train'
output_vis_dir = 'Pneumonia-YOLO/visualizations/train'  # For bounding box visualizations.

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)
os.makedirs(output_vis_dir, exist_ok=True)

resize_dims = (256, 256)

def get_yolo_bbox_from_mask(mask):
    """
    Takes a PIL grayscale image (resized to resize_dims) as input,
    returns normalized YOLO-format bounding box coordinates:
    (class, x_center, y_center, width, height), or None if no detection.
    """
    mask_np = np.array(mask)
    if mask_np.max() == 0:
        return None
    coords = cv2.findNonZero(mask_np)
    x, y, w, h = cv2.boundingRect(coords)
    H, W = mask_np.shape
    x_center = (x + w / 2) / W
    y_center = (y + h / 2) / H
    w_norm = w / W
    h_norm = h / H
    return (0, x_center, y_center, w_norm, h_norm)  # Class 0 for pneumonia.

total_images = 0
images_with_bbox = 0
bbox_areas = [] 

for file in tqdm(os.listdir(Image_dir), desc="Processing images"):
    if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    total_images += 1
    img_path = os.path.join(Image_dir, file)
    mask_path = os.path.join(Mask_dir, file)

    try:
        image = Image.open(img_path).convert("RGB")
        image_resized = image.resize(resize_dims)
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        continue

    img_out_path = os.path.join(output_img_dir, file)
    try:
        image_resized.save(img_out_path)
    except Exception as e:
        print(f"Error saving image {img_out_path}: {e}")
    
    try:
        mask = Image.open(mask_path).convert('L').resize(resize_dims)
    except Exception as e:
        print(f"Error processing mask {mask_path}: {e}")
        mask = None

    bbox = None
    if mask:
        bbox = get_yolo_bbox_from_mask(mask)

    label_file = os.path.join(output_label_dir, file.rsplit('.', 1)[0] + '.txt')
    with open(label_file, 'w') as f:
        if bbox:
            f.write(' '.join(map(str, bbox)) + '\n')
            images_with_bbox += 1
            bbox_area = bbox[3] * bbox[4]
            bbox_areas.append(bbox_area)
    
    if bbox:
        vis_image = image_resized.copy()
        draw = ImageDraw.Draw(vis_image)
        W, H = resize_dims  # (width, height)
        x_center = bbox[1] * W
        y_center = bbox[2] * H
        box_w = bbox[3] * W
        box_h = bbox[4] * H
        x1 = x_center - box_w / 2
        y1 = y_center - box_h / 2
        x2 = x_center + box_w / 2
        y2 = y_center + box_h / 2
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        vis_out_path = os.path.join(output_vis_dir, file)
        try:
            vis_image.save(vis_out_path)
        except Exception as e:
            print(f"Error saving visualization {vis_out_path}: {e}")

print(f"\nProcessed {total_images} images.")
print(f"Images with bounding boxes: {images_with_bbox}")
if bbox_areas:
    average_area = sum(bbox_areas) / len(bbox_areas)
    print(f"Average normalized bbox area: {average_area:.4f}")
else:
    print("No bounding boxes detected.")

print("\n Processing complete. YOLO images, labels, and visualizations saved.")


Overwriting yolo_detection.py


In [7]:
!python yolo_detection.py


Processing images: 100%|██████████████████| 26684/26684 [33:24<00:00, 13.31it/s]

Processed 26684 images.
Images with bounding boxes: 6012
Average normalized bbox area: 0.1751

✅ Processing complete. YOLO images, labels, and visualizations saved.
